## Anthropic APIs
### Messages 
```code
POST / v1 / messages
```
Operate using alternating `user` and `assistant` turns  
Each input message must be an object with a `role` and `content`  
The first message must always be the `user` message  

To include a system prompt, use the top-level __system__ parameter — there is no __"system"__ role for input messages in the Messages API.


### Jupyter Notebooks interface  
In general, it's important to understand that the Jupyter Notebook interface, while very convenient for **iterative development and exploration**, does not automatically persist the full execution state across sessions by default. For long-running jobs or important intermediate results, we explicitly save artifacts to disk.   

We also look into workflow tools like Papermill to parameterize and execute notebooks in a more automated fashion that saves the output.  

### Import library(-ies)

In [17]:
# Source keys from an .env
from dotenv import load_dotenv
# Source operating system resources
import os
# Access libraries for AI  
from openai import OpenAI
# Standard library for API requests
import requests
# Enable working with Unix timestamps
from datetime import datetime
# Enable working with json
import json
# Enable display of html responses
from IPython.display import display, HTML, Markdown, Latex
# Iterate paths and more
from pathlib import Path
# For basic file handling 
import shutil
# Search efficently
import re

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## Authenticate

In [18]:
# Load environment variables
load_dotenv()
api_key_pplx = os.getenv('PPLX_API_KEY')
api_key_claude = os.getenv('CLAUDE_API_KEY')

In [38]:
# Anthropic
# curl https://api.anthropic.com/v1/messages --header "x-api-key: YOUR_API_KEY" ...
base_url_claude = ""

In [19]:
# Perplexity
base_url_pplx = "https://api.perplexity.ai"

### Keep it DRY

In [20]:
# Choose model
model = "llama-3-sonar-large-32k-online"

# Create a client
client = OpenAI(api_key=api_key_pplx, base_url=base_url_pplx)

In [21]:
# Make a request
def get_completion(user_prompt, system_role, model):
    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_prompt}
    ]

    # chat completion without streaming
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    # Print the message content
    return response

In [22]:
# Accomodate common formats
def get_completion_json(system_role, user_prompt, model):
    # Note authentication is configured above

    
    # chat completion without streaming
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={"type": "json_object"}
    )
                         
    # Print the message content
    return response

In [23]:
# Print the content of the response
def summarize_message(from_the_ai):
    # Summarize the message
    print(f"The {response.choices[0].message.role} role finished \n♢ with \"{response.choices[0].finish_reason}\" at: {datetime.fromtimestamp(response.created)}")
    print(f"The model used was: {response.model}")
    print(f"♢ The message was: \n\n{response.choices[0].message.content}")

In [24]:
# Pretty print HTML response
def pretty_print_html(from_the_ai):
    display(HTML(response.choices[0].message.content))

In [25]:
# Pretty print Markdown response
def pretty_print_markdown(from_the_ai):
    display(Markdown(response.choices[0].message.content))

### Processing files

1. Use the latest models. For perplexity.ai API there is a [list](https://docs.perplexity.ai/docs/model-cards):   
2. Instructions must be clear and specific
3. Instruct at the beginning of the prompt
4. Delimit distinct parts with delimiters and use variables to chunk roles, instructions  

5. Precise, __descriptive__, with sufficient detail

In [ ]:
# Create a basic file handling function
# Previously we import os
# AND import shutil

def copy_item(src, dst):
    try:
        if os.path.isfile(src):
            # Copia o arquivo (Copy the file)
            shutil.copy(src, dst)
        elif os.path.isdir(src):
            # Copia a pasta e seu conteúdo (Copy the folder and its contents)
            shutil.copytree(src, dst)
        else:
            print(f"Item não encontrado: {src}")
            return
        
        print(f"Item copiado com sucesso: {src} -> {dst}")
    except FileExistsError:
        print(f"O item de destino já existe: {dst}")
    except PermissionError:
        print(f"Permissão negada ao copiar o item: {src}")
    except Exception as e:
        print(f"Erro ao copiar o item: {src} - {str(e)}")

def move_item(src, dst):
    try:
        # Move o arquivo ou pasta (Move the file or folder)
        shutil.move(src, dst)
        print(f"Item movido com sucesso: {src} -> {dst}")
    except FileNotFoundError:
        print(f"Item de origem não encontrado: {src}")
    except shutil.Error as e:
        print(f"Erro ao mover o item: {src} - {str(e)}")
    except Exception as e:
        print(f"Erro ao mover o item: {src} - {str(e)}")

def delete_item(item_path):
    try:
        if os.path.isfile(item_path):
            # Remove o arquivo (Remove the file)
            os.remove(item_path)
        elif os.path.isdir(item_path):
            # Remove a pasta e seu conteúdo (Remove the folder and its contents)
            shutil.rmtree(item_path)
        else:
            print(f"Item não encontrado: {item_path}")
            return
        
        print(f"Item excluído com sucesso: {item_path}")
    except PermissionError:
        print(f"Permissão negada ao excluir o item: {item_path}")
    except Exception as e:
        print(f"Erro ao excluir o item: {item_path} - {str(e)}")

In [ ]:
pretty_print_markdown(response)

### Display

In [ ]:
pretty_print_markdown(response)

### Process files

In [ ]:
# Use files as inputs for the notebook
this_cwd = Path.cwd()
print(this_cwd.name)
# vs
get_cwd = os.getcwd()
cwd_name = os.path.basename(get_cwd)
print(cwd_name)

In [ ]:
# pathlib seems cleaner, so 
csv_files = list(this_cwd.glob("*.csv"))

for file_path in csv_files:
    print(file_path)
else:
    print(f"no csvs globbed in {this_cwd.name}")



In [ ]:
# Iterate the files
for item in this_cwd.iterdir():
    print(f"a file: {item.name}")
    

In [ ]:
# but wait...is it really?
for item in this_cwd.iterdir():
    if item.is_file() and item.suffix != '':
        item_stats = item.stat()
        print(f"File: {item.name}, Suffix: {item.suffix}, Size: {item_stats.st_size} bytes")
    elif item.is_dir():
        print(f"{item.name} is a directory")
    else:
        print(f"{item.name} is not a regular file or directory")

# Use a template

## Hal
[Hal the humerous helper](https://docs.anthropic.com/en/prompt-library/hal-the-humorous-helper)


In [28]:
import random

class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]  # Hal says: "9 spaces of endless possibilities!"
        self.current_winner = None  # Hal quips: "Nobody's a winner... yet!"

    def print_board(self):
        # Hal chimes in: "Let's see this masterpiece of Xs and Os!"
        for row in [self.board[i*3:(i+1)*3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    @staticmethod
    def print_board_nums():
        # Hal jokes: "I'll give you a hint, the numbers go up!"
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')

    def available_moves(self):
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def make_move(self, square, letter):
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        # Hal exclaims: "Time to check if we have a champion!"
        row_ind = square // 3
        row = self.board[row_ind*3: (row_ind + 1) * 3]
        if all(spot == letter for spot in row):
            return True

        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        if all(spot == letter for spot in column):
            return True

        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            if all(spot == letter for spot in diagonal1):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            if all(spot == letter for spot in diagonal2):
                return True
        return False

def play(game, x_player, o_player, print_game=True):
    # Hal announces: "Let the epic battle of wits begin!"
    if print_game:
        game.print_board_nums()

    letter = 'X'  # starting letter
    # iterate while the game still has empty squares
    while game.available_moves():
        if letter == 'O':
            square = o_player.get_move(game)
        else:
            square = x_player.get_move(game)

        if game.make_move(square, letter):
            if print_game:
                print(letter + f' makes a move to square {square}')
                game.print_board()
                print('')  # just empty line

            if game.current_winner:
                if print_game:
                    print(letter + ' wins!')
                return letter

            letter = 'O' if letter == 'X' else 'X'

    if print_game:
        print("It's a tie!")

class HumanPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        valid_square = False
        val = None
        while not valid_square:
            square = input(self.letter + '\'s turn. Input move (0-8): ')
            try:
                val = int(square)
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            except ValueError:
                print('Invalid square. Try again.')
        return val

class RandomComputerPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        # Hal giggles: "Eeny, meeny, miny, moe..."
        square = random.choice(game.available_moves())
        return square

# Hal proudly presents: "The Tic Tac Toe Extravaganza!"
x_player = HumanPlayer('X')
o_player = RandomComputerPlayer('O')
t = TicTacToe()
play(t, x_player, o_player, print_game=True)

| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |


X's turn. Input move (0-8):  8


X makes a move to square 8
|   |   |   |
|   |   |   |
|   |   | X |

O makes a move to square 7
|   |   |   |
|   |   |   |
|   | O | X |



X's turn. Input move (0-8):  0


X makes a move to square 0
| X |   |   |
|   |   |   |
|   | O | X |

O makes a move to square 3
| X |   |   |
| O |   |   |
|   | O | X |



X's turn. Input move (0-8):  4


X makes a move to square 4
| X |   |   |
| O | X |   |
|   | O | X |

X wins!


'X'

In [30]:
"""
Tic-Tac-Toe Module

An implementation of a simple game
Includes game logic, player classes, and a main game loop.

Classes:
    TicTacToe: Represents the game board and rules.
    HumanPlayer: Represents a human player in the game which is beatable.
    AIPlayer: Represents an AI player using the minimax algorithm which should only ever allow a tie.

Functions:
    play(game, x_player, o_player, print_game=True): Runs the game.
"""

import random

class TicTacToe:
    """
    A class to represent the game of tic-tac-toe.
    Playing on 3x3 grid, children of all ages, alternately place their markers ("X" or "O")
    If a player succeeds in placing three in a row, they win, and the game begins again.

    Attributes:
        board (list): nine strings representing the field of play
        current_winner (str): the symbol of the current winner
    """
    def __init__(self):
        """
        Initialize the board and set current winner to none
        """
        self.board = [' ' for _ in range(9)]  # Hal says: "9 spaces of endless possibilities!"
        self.current_winner = None  # Hal quips: "Nobody's a winner... yet!"

    def print_board(self):
        # Hal chimes in: "Let's see this masterpiece of Xs and Os!"
        for row in [self.board[i*3:(i+1)*3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    @staticmethod
    def print_board_nums():
        # Hal jokes: "I'll give you a hint, the numbers go up!"
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')

    def available_moves(self):
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def make_move(self, square, letter):
        """
        Place a marker 'on' the specified square of the board.

        Args:
            square (int): An index of the square to place a marker (0-8).
            letter (str): The player's marker ('X' or 'O').

        Returns:
            bool: True if the move was successful, False otherwise.
        """
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        """
        Check if the current move is the winning move.

        Args:
            square (int): The index of the last move made.
            letter (str): The marker ('X' or 'O') of the current player.

        Returns:
            bool: True if the current move wins, False if not.
        """
        # Hal exclaims: "Time to check if we have a champion!"
        row_ind = square // 3
        row = self.board[row_ind*3: (row_ind + 1) * 3]
        if all(spot == letter for spot in row):
            return True

        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        if all(spot == letter for spot in column):
            return True

        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            if all(spot == letter for spot in diagonal1):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            if all(spot == letter for spot in diagonal2):
                return True
        return False

def play(game, x_player, o_player, print_game=True):
    # Hal announces: "Let the epic battle of wits begin!"
    if print_game:
        game.print_board_nums()

    letter = 'X'  # starting letter
    # iterate while the game still has empty squares
    while game.available_moves():
        if letter == 'O':
            square = o_player.get_move(game)
        else:
            square = x_player.get_move(game)

        if game.make_move(square, letter):
            if print_game:
                print(letter + f' makes a move to square {square}')
                game.print_board()
                print('')  # just empty line

            if game.current_winner:
                if print_game:
                    print(letter + ' wins!')
                return letter

            letter = 'O' if letter == 'X' else 'X'

    if print_game:
        print("It's a tie!")

class HumanPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        valid_square = False
        val = None
        while not valid_square:
            square = input(self.letter + '\'s turn. Input move (0-8): ')
            try:
                val = int(square)
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            except ValueError:
                print('Invalid square. Try again.')
        return val

class RandomComputerPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        # Hal giggles: "Eeny, meeny, miny, moe..."
        square = random.choice(game.available_moves())
        return square



class AIPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        if len(game.available_moves()) == 9:
            square = random.choice(game.available_moves())  # randomly choose one
        else:
            # get the square based off the minimax algorithm
            square = self.minimax(game, self.letter)['position']
        return square

    def minimax(self, state, player):
        max_player = self.letter  # yourself
        other_player = 'O' if player == 'X' else 'X'

        # first we want to check if the previous move is a winner
        if state.current_winner == other_player:
            return {'position': None, 'score': 1 * (len(state.available_moves()) + 1) if other_player == max_player else -1 * (
                        len(state.available_moves()) + 1)}
        elif not state.available_moves():  # no empty squares
            return {'position': None, 'score': 0}

        if player == max_player:
            best = {'position': None, 'score': -float('inf')}  # each score should maximize
        else:
            best = {'position': None, 'score': float('inf')}  # each score should minimize

        for possible_move in state.available_moves():
            state.make_move(possible_move, player)
            sim_score = self.minimax(state, other_player)  # simulate a game after making that move

            # undo move
            state.board[possible_move] = ' '
            state.current_winner = None
            sim_score['position'] = possible_move  # this represents the move optimal next move

            if player == max_player:  # X is max player
                if sim_score['score'] > best['score']:
                    best = sim_score
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score
        return best

# Hal proudly presents: "The Tic Tac Toe Extravaganza!"
x_player = HumanPlayer('X')
# o_player = RandomComputerPlayer('O')

# With this:
o_player = AIPlayer('O')
t = TicTacToe()
play(t, x_player, o_player, print_game=True)

| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |


X's turn. Input move (0-8):  8


X makes a move to square 8
|   |   |   |
|   |   |   |
|   |   | X |

O makes a move to square 4
|   |   |   |
|   | O |   |
|   |   | X |



X's turn. Input move (0-8):  1


X makes a move to square 1
|   | X |   |
|   | O |   |
|   |   | X |

O makes a move to square 0
| O | X |   |
|   | O |   |
|   |   | X |



...

X makes a move to square 6
| O | X |   |
|   | O |   |
| X |   | X |

O makes a move to square 7
| O | X |   |
|   | O |   |
| X | O | X |



X's turn. Input move (0-8):  5


X makes a move to square 5
| O | X |   |
|   | O | X |
| X | O | X |

O makes a move to square 2
| O | X | O |
|   | O | X |
| X | O | X |



X's turn. Input move (0-8):  3


X makes a move to square 3
| O | X | O |
| X | O | X |
| X | O | X |

It's a tie!


In [ ]:
# previously we imported from IPython.display import display, HTML

message_content = message.content

# Extract the text from the TextBlock
text_block = message.content[0]
text = text_block.text
print(text)